In [7]:
import os
import collections
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import pickle
import csv

In [8]:
def read_opcodes_from_file(filepath):
    with open(filepath, 'r') as file:
        opcodes = file.read().split()
    return opcodes

def create_opcode_frequency(opcodes):
    return collections.Counter(opcodes)

def process_directory(directory_path):
    files = os.listdir(directory_path)
    dataset = []
    for filename in files:
        filepath = os.path.join(directory_path, filename)
        opcodes = read_opcodes_from_file(filepath)
        frequency = create_opcode_frequency(opcodes)
        dataset.append(frequency)
    return dataset, files

In [9]:
# Paths to your malware and goodware folders
malware_dir = '/home/ubuntu/Desktop/Dataset/Our-Datasets/IoT/OpCode/OpCode/malware'
goodware_dir = '/home/ubuntu/Desktop/Dataset/Our-Datasets/IoT/OpCode/OpCode/goodware'

# Process directories to create datasets
malware_data, malware_files = process_directory(malware_dir)
goodware_data, goodware_files = process_directory(goodware_dir)

# Create labels
y_malware = [1] * len(malware_data)  # Label malware as 1
y_goodware = [0] * len(goodware_data)  # Label goodware as 0

In [13]:
# Combine the datasets and labels
data = malware_data + goodware_data
labels = y_malware + y_goodware
files = malware_files + goodware_files

# Vectorize the dataset
vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(data)
y = labels

# Convert to DataFrame
df = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())
df['label'] = y
df['filename'] = files

# Save DataFrame to CSV
df.to_csv('opcode_dataset_refined.csv', index=False)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a machine learning model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100}%')

# Save the model to a file
with open('logistic_regression_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
    
# Save the vectorizer to a file
with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

Accuracy: 99.02912621359224%
